In [1]:
# import main libraries
import json
import requests
import pandas as pd
import numpy as np

In [2]:
# import all datasets
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json")
incidents_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones.json")
distances_stations_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/barrios.json")
ntas_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json")
stations_db = json.loads(response.text)

In [8]:
df_ntas = pd.read_csv("dataset/neighborhoods.csv", index_col="nta")
df_distance_stations_ntas = pd.read_csv("dataset/df_distance_stations_neighborhoods.csv", index_col="Unnamed: 0")

In [10]:
vehicle_types = ["engine", "ladder", "rescue", "squad", "hazardous"]

In [11]:
# create dictionaries for incident average durations for both shifts
vehicle_incident_average_duration_first_shift = {}
vehicle_incident_average_duration_second_shift = {}

# iterate all vehicle types
for vehicle_type in vehicle_types:
    # define index names
    incident_average_duration_first_shift = vehicle_type + "_incident_average_duration_first_shift"
    incident_average_duration_second_shift = vehicle_type + "_incident_average_duration_second_shift"

    # get just the neighborhood where some incidents happened (where we have an average incident duration)
    non_zero_first_shift = df_ntas[df_ntas[incident_average_duration_first_shift] != 0]
    # do the mean of the incident durations for that vehicle types (between all neighborhoods)
    vehicle_incident_average_duration_first_shift[vehicle_type] = np.mean(non_zero_first_shift[incident_average_duration_first_shift])

    non_zero_second_shift = df_ntas[df_ntas[incident_average_duration_second_shift] != 0]
    vehicle_incident_average_duration_second_shift[vehicle_type] = np.mean(non_zero_second_shift[incident_average_duration_second_shift])

In [12]:
# average incident durations in the first shift
vehicle_incident_average_duration_first_shift

{'engine': 1985.5373439159384,
 'ladder': 1906.7642481828568,
 'rescue': 2271.5054389954926,
 'squad': 2187.5583007060777,
 'hazardous': 1611.7435902715163}

In [13]:
# average incident durations in the second shift
vehicle_incident_average_duration_second_shift

{'engine': 1647.5697278471393,
 'ladder': 1628.2509015419084,
 'rescue': 1699.3350584143634,
 'squad': 1610.8915117386066,
 'hazardous': 1612.058942803136}

In [15]:
# create two dictionaries (one for each shift) for the serviceability o the stations
df_station_serviceability_first_shift = {}
df_station_serviceability_second_shift = {}

# iterate all vehicle types
for vehicle_type in vehicle_types:
    # create datframes of the same shape of the nta-stations distances, all false 
    df_station_serviceability_first_shift[vehicle_type] = df_distance_stations_ntas.copy()
    df_station_serviceability_first_shift[vehicle_type].loc[:] = False
    df_station_serviceability_second_shift[vehicle_type] = df_distance_stations_ntas.copy()
    df_station_serviceability_second_shift[vehicle_type].loc[:] = False

In [17]:
# iterate all vehicle types
for vehicle_type in vehicle_types:
    # get the right dataframe for the vehicle
    vehicle_serviceability_first_shift = df_station_serviceability_first_shift[vehicle_type]
    vehicle_serviceability_second_shift = df_station_serviceability_second_shift[vehicle_type]
    
    # interate together the neigborhoods on nta-stations distances dataframe and neighborhoods dataframe
    for (nta, distances), (nta, nta_data) in zip(df_distance_stations_ntas.iterrows(), df_ntas.iterrows()):
        # iterate stations' distances from the neighborhood
        for station, distance_st_nta in distances.items():
            # check if we have an average incident duration for that neighborhood
            if nta_data[vehicle_type + "_incident_average_duration_first_shift"] > 0:
                # check if the distance between the nta and the station is less then half the average incident duration for that neighborhood
                if distance_st_nta < (nta_data[vehicle_type + "_incident_average_duration_first_shift"] / 2):
                    # set the neighborhood as servable by that station
                    vehicle_serviceability_first_shift.loc[nta, station] = True
            else:
                # check if the distance between the nta and the station is less then half the average incident duration for all neighborhoods
                if distance_st_nta < vehicle_incident_average_duration_first_shift[vehicle_type] / 2:
                    # set the neighborhood as servable by that station
                    vehicle_serviceability_first_shift.loc[nta, station] = True

            # do the same operations for the second shift
            if nta_data[vehicle_type + "_incident_average_duration_second_shift"] > 0:
                if distance_st_nta < nta_data[vehicle_type + "_incident_average_duration_second_shift"] / 2:
                    vehicle_serviceability_second_shift.loc[nta, station] = True
            else:
                if distance_st_nta < vehicle_incident_average_duration_second_shift[vehicle_type] / 2:
                    vehicle_serviceability_second_shift.loc[nta, station] = True

In [21]:
for df in df_station_serviceability_first_shift:
    df_station_serviceability_first_shift[df].to_csv("dataset/serviceability/first_shift/" + df + ".csv")

for df in df_station_serviceability_second_shift:
    df_station_serviceability_second_shift[df].to_csv("dataset/serviceability/second_shift/" + df + ".csv")